In [1]:
import os
import sys
p = os.path.abspath('..')
if p not in sys.path:
    sys.path.append(p)
from src.table_utils import make_table
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
def get_results(data_type, timestamps):
    num_data = len(timestamps)
    data_dict_list = []
    for timestamp in timestamps:
        data_dict_list.append(pickle.load( open( "../data/midprice_prediction_{}_{}.pkl".format(data_type, timestamp), "rb" ) ))
        
    columns_overall = ("Accuracy", "Macro F1", "Macro Precision", "Macro Recall")
    funs_overall = (accuracy_score, lambda yt, yp: f1_score(yt, yp, average="macro"), lambda yt, yp: precision_score(yt, yp, average="macro"), lambda yt, yp: recall_score(yt, yp, average="macro"))
    df = pd.DataFrame(columns=("Score", "RNN model", "DeepLOB"))
    idx = 0
    for i in range(num_data):
        true_labels = data_dict_list[i]["test_labels"].argmax(axis=-1)
        alpha = data_dict_list[i]["alpha"]
        lt_rnn = data_dict_list[i]["simulation_prediction"]
        rnn_labels = np.where(np.mean(lt_rnn, axis=-1) > alpha, 1, 0)
        rnn_labels = np.where(np.mean(lt_rnn, axis=-1) < -alpha, -1, rnn_labels)
        rnn_labels = rnn_labels + 1
        deeplob_labels = data_dict_list[i]["deeplob_test_prediction"].argmax(axis=-1)
        for k, fun in zip(columns_overall, funs_overall):
            rnn_score = fun(true_labels, rnn_labels)
            deeplob_score = fun(true_labels, deeplob_labels)
            df.loc[idx] = [k, rnn_score, deeplob_score]
            idx += 1
            
    table_filename = f"table_prediction_{data_type}.tex"
    grouped, code = make_table(df,
                         ["Score"],
                         ["RNN model", "DeepLOB"],
                         highorlow="high"
                         )
    with open(table_filename, "w") as fp:
        fp.write(code)
    return df, grouped, code

In [3]:
data_type = "nasdaq"
timestamps = ["2022_06_01_18_01", "2022_06_03_18_12_30", "2022_06_03_18_12_43", "2022_06_03_18_12_47", "2022_06_03_18_13_39"]
df, grouped, code = get_results(data_type, timestamps)

{'Accuracy': (0.6292775607205151, True), 'Macro F1': (0.6365928567131176, True), 'Macro Precision': (0.6517356634768209, True), 'Macro Recall': (0.6298632613434902, True)}


In [4]:
grouped

RNN model             DeepLOB          
                     mean       std      mean       std
Score                                                  
Accuracy         0.636740  0.007462  0.630980  0.004944
Macro F1         0.643783  0.007190  0.631821  0.005528
Macro Precision  0.659907  0.008171  0.633901  0.007027
Macro Recall     0.637216  0.007353  0.631193  0.004569

In [5]:
data_type = "mc"
timestamps = ["2022_06_01_18_01", "2022_06_03_18_12_23", "2022_06_03_18_13_21", "2022_06_03_18_13_24", "2022_06_03_18_13_34"]
df, grouped, code = get_results(data_type, timestamps)

{'Accuracy': (0.6467107234120291, True), 'Macro F1': (0.6468589595793838, True), 'Macro Precision': (0.6541014857203205, True), 'Macro Recall': (0.6458289182083513, True)}


In [6]:
grouped

RNN model             DeepLOB          
                     mean       std      mean       std
Score                                                  
Accuracy         0.615620  0.002109  0.653000  0.006289
Macro F1         0.618510  0.002072  0.653370  0.006511
Macro Precision  0.622900  0.002986  0.659966  0.005864
Macro Recall     0.616212  0.001823  0.652287  0.006458